In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jan-publica-pred/submission.csv
/kaggle/input/census-data-for-godaddy/ACSST5Y2017.S0101-Data.csv
/kaggle/input/census-data-for-godaddy/ACSST5Y2018.S0101-Table-Notes.txt
/kaggle/input/census-data-for-godaddy/ACSST5Y2019.S0101-Table-Notes.txt
/kaggle/input/census-data-for-godaddy/ACSST5Y2021.S0101-Table-Notes.txt
/kaggle/input/census-data-for-godaddy/ACSST5Y2019.S0101-Column-Metadata.csv
/kaggle/input/census-data-for-godaddy/ACSST5Y2020.S0101-Data.csv
/kaggle/input/census-data-for-godaddy/ACSST5Y2021.S0101-Column-Metadata.csv
/kaggle/input/census-data-for-godaddy/ACSST5Y2018.S0101-Column-Metadata.csv
/kaggle/input/census-data-for-godaddy/ACSST5Y2017.S0101-Column-Metadata.csv
/kaggle/input/census-data-for-godaddy/ACSST5Y2020.S0101-Column-Metadata.csv
/kaggle/input/census-data-for-godaddy/ACSST5Y2021.S0101-Data.csv
/kaggle/input/census-data-for-godaddy/ACSST5Y2017.S0101-Table-Notes.txt
/kaggle/input/census-data-for-godaddy/ACSST5Y2019.S0101-Data.csv
/kaggle/input/census-data-

In [2]:
train = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/train.csv')
reaveal_test = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/revealed_test.csv')
train = pd.concat([train, reaveal_test]).sort_values(by=['cfips','first_day_of_month']).reset_index(drop=True)

In [3]:
march_pred = pd.read_csv('/kaggle/input/godday-inference-20-length/gru_xgb_blending_3th_submission.csv')
april_pred = pd.read_csv('/kaggle/input/godday-inference-20-length/gru_xgb_blending_4th_submission.csv')
may_pred = pd.read_csv('/kaggle/input/godday-inference-20-length/gru_xgb_blending_5th_submission.csv')

In [4]:
march_pred.head(10)

,row_id,microbusiness_density
0,1001_2022-11-01,3.335074
1,1001_2022-12-01,3.335074
2,1001_2023-01-01,3.335074
3,1001_2023-02-01,3.335074
4,1001_2023-03-01,3.335074
5,1001_2023-04-01,3.335074
6,1001_2023-05-01,3.335074
7,1001_2023-06-01,3.335074
8,1003_2022-11-01,7.964891
9,1003_2022-12-01,7.964891


In [5]:
march_pred['cfips'] = march_pred['row_id'].apply(lambda x: int(x.split('_')[0]))
april_pred['cfips'] = april_pred['row_id'].apply(lambda x: int(x.split('_')[0]))
may_pred['cfips'] = may_pred['row_id'].apply(lambda x: int(x.split('_')[0]))

In [6]:
dt_3 = march_pred.groupby('cfips')['microbusiness_density'].last().to_dict()
dt_4 = april_pred.groupby('cfips')['microbusiness_density'].last().to_dict()
dt_5 = may_pred.groupby('cfips')['microbusiness_density'].last().to_dict()

In [7]:
dt_3[1003],dt_4[1003],dt_5[1003]

(7.96489149658909, 7.969922236882738, 8.023974621711911)

In [8]:
dt_3[28055],dt_4[28055],dt_5[28055]

(0.102880658436214, 0.102880658436214, 0.102880658436214)

In [9]:
jan_pred = pd.read_csv('/kaggle/input/jan-publica-pred/submission.csv')
jan_pred['cfips'] = jan_pred['row_id'].apply(lambda x: int(x.split('_')[0]))
jan_pred['time'] = jan_pred['row_id'].apply(lambda x: x.split('_')[1])
jan_pred = jan_pred.loc[jan_pred.time=='2023-01-01'].reset_index(drop=True)
jan_pred.head()

,row_id,microbusiness_density,cfips,time
0,1001_2023-01-01,3.337234,1001,2023-01-01
1,1003_2023-01-01,7.962157,1003,2023-01-01
2,1005_2023-01-01,1.245311,1005,2023-01-01
3,1007_2023-01-01,1.292135,1007,2023-01-01
4,1009_2023-01-01,1.827393,1009,2023-01-01


In [10]:
dt_1 = jan_pred[['microbusiness_density','cfips']].set_index('cfips').to_dict()['microbusiness_density']

In [11]:
dt_1[1001]

3.3372338989153425

# submission

In [12]:
submission = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/sample_submission.csv')
submission['cfips'] = submission['row_id'].apply(lambda x: int(x.split('_')[0]))
submission['time'] = submission['row_id'].apply(lambda x: x.split('_')[1])

In [13]:
submission.loc[submission.time=='2023-01-01','microbusiness_density']=submission.loc[submission.time=='2023-01-01','cfips'].map(dt_1)
submission.loc[submission.time=='2023-01-01','microbusiness_density'][:5],list(dt_1.values())[:5]

(6270    3.337234
 6271    7.962157
 6272    1.245311
 6273    1.292135
 6274    1.827393
 Name: microbusiness_density, dtype: float64,
 [3.3372338989153425,
  7.962157154487522,
  1.245311327831958,
  1.292134831460674,
  1.827393199265503])

In [14]:
submission.loc[submission.time=='2023-02-01','microbusiness_density']=submission.loc[submission.time=='2023-02-01','cfips'].map(dt_1)
submission.loc[submission.time=='2023-02-01','microbusiness_density'][:5],list(dt_1.values())[:5]

(9405    3.337234
 9406    7.962157
 9407    1.245311
 9408    1.292135
 9409    1.827393
 Name: microbusiness_density, dtype: float64,
 [3.3372338989153425,
  7.962157154487522,
  1.245311327831958,
  1.292134831460674,
  1.827393199265503])

In [15]:
submission.loc[submission.time=='2023-03-01','microbusiness_density']=submission.loc[submission.time=='2023-03-01','cfips'].map(dt_3)
submission.loc[submission.time=='2023-03-01','microbusiness_density'][:5],list(dt_3.values())[:5]

(12540    3.335074
 12541    7.964891
 12542    1.239510
 12543    1.290449
 12544    1.821022
 Name: microbusiness_density, dtype: float64,
 [3.3350735856699223,
  7.96489149658909,
  1.2395098774693674,
  1.2904494382022471,
  1.821021658812858])

In [16]:
submission.loc[submission.time=='2023-04-01','microbusiness_density']=submission.loc[submission.time=='2023-04-01','cfips'].map(dt_4)
submission.loc[submission.time=='2023-04-01','microbusiness_density'][:5],list(dt_4.values())[:5]

(15675    3.344953
 15676    7.969922
 15677    1.238510
 15678    1.288090
 15679    1.827637
 Name: microbusiness_density, dtype: float64,
 [3.3449525181151265,
  7.969922236882738,
  1.2385096274068517,
  1.2880898876404494,
  1.827636556713347])

In [17]:
submission.loc[submission.time=='2023-05-01','microbusiness_density']=submission.loc[submission.time=='2023-05-01','cfips'].map(dt_5)
submission.loc[submission.time=='2023-05-01','microbusiness_density'][:5],list(dt_5.values())[:5]

(18810    3.361267
 18811    8.023975
 18812    1.243811
 18813    1.295787
 18814    1.837083
 Name: microbusiness_density, dtype: float64,
 [3.3612673837706466,
  8.023974621711911,
  1.2438109527381846,
  1.2957865168539326,
  1.837083250370567])

In [18]:
submission.loc[submission.time=='2023-06-01','microbusiness_density']=submission.loc[submission.time=='2023-06-01','cfips'].map(dt_5)
submission.loc[submission.time=='2023-06-01','microbusiness_density'][:5],list(dt_5.values())[:5]

(21945    3.361267
 21946    8.023975
 21947    1.243811
 21948    1.295787
 21949    1.837083
 Name: microbusiness_density, dtype: float64,
 [3.3612673837706466,
  8.023974621711911,
  1.2438109527381846,
  1.2957865168539326,
  1.837083250370567])

In [19]:
submission['outliers']=submission.groupby('cfips')['microbusiness_density'].transform(lambda x:int(abs(x[3:4].values-x[5:6].values)>0.02*x[3:4].values))

In [20]:
submission['outliers'].sum()/8

150.0

In [21]:
BASE='/kaggle/input/census-data-for-godaddy/'
#21
COLS = ['GEO_ID','S0101_C01_026E']
df2021 = pd.read_csv(BASE+'ACSST5Y2021.S0101-Data.csv',usecols=COLS)
df2021 = df2021.iloc[1:]
df2021['S0101_C01_026E'] = df2021['S0101_C01_026E'].astype('int')
df2021 = df2021.rename(columns={'S0101_C01_026E':'census2021'})

#20
df2020 = pd.read_csv(BASE+'ACSST5Y2020.S0101-Data.csv',usecols=COLS)
df2020 = df2020.iloc[1:]
df2020['S0101_C01_026E'] = df2020['S0101_C01_026E'].astype('int')
df2020 = df2020.rename(columns={'S0101_C01_026E':'census2020'})
#19
df2019 = pd.read_csv(BASE+'ACSST5Y2019.S0101-Data.csv',usecols=COLS)
df2019 = df2019.iloc[1:]
df2019['S0101_C01_026E'] = df2019['S0101_C01_026E'].astype('int')
df2019 = df2019.rename(columns={'S0101_C01_026E':'census2019'})
#18
df2018 = pd.read_csv(BASE+'ACSST5Y2018.S0101-Data.csv',usecols=COLS)
df2018 = df2018.iloc[1:]
df2018['S0101_C01_026E'] = df2018['S0101_C01_026E'].astype('int')
df2018 = df2018.rename(columns={'S0101_C01_026E':'census2018'})
#17
df2017 = pd.read_csv(BASE+'ACSST5Y2017.S0101-Data.csv',usecols=COLS)
df2017 = df2017.iloc[1:]
df2017['S0101_C01_026E'] = df2017['S0101_C01_026E'].astype('int')
df2017 = df2017.rename(columns={'S0101_C01_026E':'census2017'})
census_years = df2021.merge(df2020,on='GEO_ID',how='left')
census_years = census_years.merge(df2019,on='GEO_ID',how='left')
census_years = census_years.merge(df2018,on='GEO_ID',how='left')
census_years = census_years.merge(df2017,on='GEO_ID',how='left')
census_years['cfips'] = census_years.GEO_ID.apply(lambda x:int(x.split('US')[-1]))
census_years = census_years.drop('GEO_ID',axis=1)
census_years.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,census2021,census2020,census2019,census2018,census2017,cfips
0,44438,42496,42175.0,41831.0,41527.0,1001
1,178105,171296,166595.0,162430.0,158361.0,1003
2,19995,19804,20054.0,20346.0,20687.0,1005
3,17800,17790,17862.0,17868.0,17852.0,1007
4,45201,44383,44292.0,44177.0,44102.0,1009


In [22]:
train = train.merge(census_years,on='cfips',how='left')
train["year"] = train["first_day_of_month"].apply(lambda x:int(x.split('-')[0]))
#smooth the md
for year in train.year.unique():
    if year==2019:
        train.loc[train.year==year,'microbusiness_density'] = train.loc[train.year==year,'microbusiness_density']*train.loc[train.year==year,'census2017']/train.loc[train.year==year,'census2021']
    if year==2020:
        train.loc[train.year==year,'microbusiness_density'] = train.loc[train.year==year,'microbusiness_density']*train.loc[train.year==year,'census2018']/train.loc[train.year==year,'census2021']
    elif year==2021:
        train.loc[train.year==year,'microbusiness_density'] = train.loc[train.year==year,'microbusiness_density']*train.loc[train.year==year,'census2019']/train.loc[train.year==year,'census2021']
    elif year==2022:
        train.loc[train.year==year,'microbusiness_density'] = train.loc[train.year==year,'microbusiness_density']*train.loc[train.year==year,'census2020']/train.loc[train.year==year,'census2021']

In [23]:
outlier_cfips=submission.loc[submission.outliers==1,'cfips'].unique()

In [24]:
import matplotlib.pyplot as plt
# for i in outlier_cfips:
#     print('cfips',i)
#     print(train.loc[train.cfips==i,['first_day_of_month','microbusiness_density']][-4:])
#     print(submission.loc[submission.cfips==i,['time','microbusiness_density']][2:-1])
#     train.loc[train.cfips==i,['first_day_of_month','microbusiness_density']].plot(y='microbusiness_density')
#     plt.show()

In [25]:
over_high_0005=[40143,47033,47187,49043,54045]
over_high_001=[10001,13307,16021,20063,20083,39063,40017,53065]
over_high_002=[13271,35041,47031,47175,]
over_high_003=[17087,18043,20153,23027,27035,47103,48011]
over_high_004=[39041,56033]
over_low_001=[13285,21119,21133,21159,21193,28021,38005,42003,46007,54059]
over_low_002=[5141,6005,21053,21109,21201,21203,32017,48207]
over_low_003=[21131,21147]
over_low_004=[20011,]
remain_last=[31165,30069,30079,31077,31113,46069,46137,48125,48311,49009,51081]

In [26]:
revised = [(over_high_0005,-0.005),(over_high_001,-0.01),(over_high_002,-0.02),(over_high_003,-0.03),
          (over_high_004,-0.04),
           (over_low_001,0.01),(over_low_002,0.02),(over_low_003,0.03),(over_low_004,0.04)]

In [27]:
for i in revised:
    flag = 1+i[1]
    print(flag)
    revised_idex = (submission.cfips.isin(i[0][:1]))&(submission.time.isin(['2023-03-01','2023-04-01','2023-05-01']))
    submission.loc[revised_idex,'microbusiness_density']=flag*submission.loc[revised_idex,'microbusiness_density']

0.995
0.99
0.98
0.97
0.96
1.01
1.02
1.03
1.04


In [28]:
remain_last = [31165,30069,30079,31077,31113,46069,46137,48125,48311,49009,51081]
remain_idx = (submission.cfips.isin(remain_last))&(submission.time.isin(['2023-03-01','2023-04-01','2023-05-01']))

In [29]:
remain_dt=train.loc[(train.cfips.isin(remain_last))&(train.first_day_of_month.isin(['2022-12-01'])),['cfips','microbusiness_density']].set_index('cfips').to_dict()['microbusiness_density']

In [30]:

submission.loc[remain_idx,'microbusiness_density']=submission.loc[remain_idx,['cfips','microbusiness_density']]['cfips'].map(remain_dt)

In [31]:
submission.loc[submission.row_id=='1001_2023-03-01','microbusiness_density']+=0.005

In [32]:
cifp=46137
print(dt_3[cifp],dt_4[cifp],dt_5[cifp])
submission.loc[submission.cfips==cifp][:]

0.3833705357142857 0.3805803571428571 0.3950892857142857


,row_id,microbusiness_density,cfips,time,outliers
2423,46137_2022-11-01,3.817671,46137,2022-11-01,1
5558,46137_2022-12-01,3.817671,46137,2022-12-01,1
8693,46137_2023-01-01,0.390625,46137,2023-01-01,1
11828,46137_2023-02-01,0.390625,46137,2023-02-01,1
14963,46137_2023-03-01,0.390625,46137,2023-03-01,1
18098,46137_2023-04-01,0.390625,46137,2023-04-01,1
21233,46137_2023-05-01,0.390625,46137,2023-05-01,1
24368,46137_2023-06-01,0.395089,46137,2023-06-01,1


In [33]:
submission.drop(['cfips','time','outliers'],axis=1,inplace=True)

In [34]:
submission.to_csv('submission.csv',index=False)

In [35]:
(submission.microbusiness_density>900).sum(),(submission.microbusiness_density<0).sum(),(submission.microbusiness_density.isna()).sum()

(0, 0, 0)

In [36]:
submission

,row_id,microbusiness_density
0,1001_2022-11-01,3.817671
1,1003_2022-11-01,3.817671
2,1005_2022-11-01,3.817671
3,1007_2022-11-01,3.817671
4,1009_2022-11-01,3.817671
...,...,...
25075,56037_2023-06-01,2.937678
25076,56039_2023-06-01,26.642548
25077,56041_2023-06-01,4.024315
25078,56043_2023-06-01,3.116833


In [37]:
6005

6005